# Tutorial

**Op het moment is het _vera-testframework_ alleen compatibel met _pyspark_.**

In [1]:
from vera_testframework.pyspark import ReferentiedataTest
from pyspark.sql import SparkSession
from testframework.dataquality import DataFrameTester

In [2]:
spark = SparkSession.builder.appName("vera_testframework").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/12 15:10:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


**Hieronder wordt een voorbeeld DataFrame gemaakt m.b.t. ruimten, waarvan we gaan testen of de waardes voldoen aan de VERA-standaard.**

In [3]:
ruimten = [
    (1, "LOG", "Loggia"),
    (2, "WOO", "Woonkamer"),
    (3, "BAD", "Badruimte"),
    (4, "BAD", "Badkamer"),
    (5, None, "Kelder"),
    (6, "SLA", None),
]

ruimten_df = spark.createDataFrame(ruimten, ["id", "code", "naam"])

**We maken gebruik van de `DataFrameTester` van het _pyspark-testframework_ om onze testresultaten in bij te houden.**

In [4]:
testframework = DataFrameTester(
    df=ruimten_df,
    primary_key="id",
    spark=spark,
)

**Door middel van de `ReferentiedataTest` kunnen we testen of een kolom voldoet aan de VERA-standaard m.b.t. Referentiedata.**

In [5]:
testframework.test(
    col="code",
    test=ReferentiedataTest(
        soort="RUIMTEDETAILSOORT",
        attribuut="Code",
        release="latest",  # standaard is latest, maar kan ook een specifieke versie zijn zoals v4.1.240419
    ),
    nullable=False,  # of een waarde leeg mag zijn. Dit is aan de gebruiker
).show()

+---+----+-------------------+
| id|code|code__VERAStandaard|
+---+----+-------------------+
|  1| LOG|               true|
|  2| WOO|               true|
|  3| BAD|               true|
|  4| BAD|               true|
|  5|NULL|              false|
|  6| SLA|               true|
+---+----+-------------------+



In [6]:
testframework.test(
    col="naam",
    test=ReferentiedataTest(
        soort="RUIMTEDETAILSOORT",
        attribuut="Naam",
    ),
    nullable=True,
).show()

+---+---------+-------------------+
| id|     naam|naam__VERAStandaard|
+---+---------+-------------------+
|  1|   Loggia|               true|
|  2|Woonkamer|               true|
|  3|Badruimte|              false|
|  4| Badkamer|               true|
|  5|   Kelder|               true|
|  6|     NULL|               true|
+---+---------+-------------------+



**De resultaten van de testen zijn te vinden in de `.results` attribuut van de `DataFrameTester`.**

In [7]:
testframework.results.show()

+---+-------------------+-------------------+
| id|code__VERAStandaard|naam__VERAStandaard|
+---+-------------------+-------------------+
|  1|               true|               true|
|  2|               true|               true|
|  3|               true|              false|
|  4|               true|               true|
|  5|              false|               true|
|  6|               true|               true|
+---+-------------------+-------------------+



**Voor meer informatie over hoe het _pyspark-testframework_ te gebruiken, raadpleeg de documentatie op [hun Github](https://github.com/woonstadrotterdam/pyspark-testframework)**

In [8]:
spark.stop()